In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read the data
df = pd.read_csv('HLE_DB22_Dataset.csv')
df.head()

,Unnamed: 0,Link,Headline,Category,Subtitle,Keywords
0,0,https://www.dagbladet.no/nyheter/kaotisk-nytta...,kaotisk nyttarsnatt ute av kontroll,nyheter,Det har vært en hektisk nyttårsnatt for politi...,nyheter
1,1,https://www.dagbladet.no/tema/fra-singel-til-d...,fra singel til damemagnet,tema,Ekspertenes beste råd til hvordan du blir attr...,"kjæreste,dating,singel"
2,2,https://www.dagbladet.no/tema/strom-nytt-fra-j...,strom nytt fra januar,tema,"Nye boligregler, bedre jobbpensjon og dyrere f...","metadz,økonomi,pensjon,skatt"
3,3,https://www.dagbladet.no/tema/anders-kvitt-mag...,anders kvitt magefettet i superfart 1,tema,"- Hvis folk bare visste hvor enkelt det er, ha...","magefett,vektnedgang,kosthold,styrketrening,"
4,4,https://www.dagbladet.no/nyheter/taus-om-overg...,taus om overgrepsdom,nyheter,Trump ønsket «henne alt godt» i fjor sommer. N...,"donald trump,ghislaine maxwell,utenriks,nyhete..."


In [2]:
from transformers import pipeline

ner = pipeline(task='ner', 
               model='saattrupdan/nbailab-base-ner-scandi', 
               aggregation_strategy='first')

result = ner('Borghild kjøper seg inn i Bunnpris')
result

c:\Users\mathi\Prosjekter\Barcelona\HLE\HLE-Project\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'entity_group': 'PER',
  'score': 0.9812571,
  'word': 'Borghild',
  'start': 0,
  'end': 8},
 {'entity_group': 'ORG',
  'score': 0.9740986,
  'word': 'Bunnpris',
  'start': 26,
  'end': 34}]

In [3]:
# Check rows with missing values
df.isnull().sum()

Unnamed: 0      0
Link          180
Headline      180
Category      180
Subtitle      721
Keywords       59
dtype: int64

In [4]:
df = df.dropna()

In [18]:
def ner_lambda(x: str):
    try:
        pred = ner(x)
        result = {}
        if pred is not None:
            for d in pred:
                result[d['word']] = d['entity_group']
        return result
    except Exception as e:
        print(f"Exception: {e}\nCaused by: {x}")
        return {}

In [7]:
df['entities'] = df['Subtitle'].apply(lambda x: ner_lambda(x))

# Write the result to dataframe 'Data_NER.csv'
df.to_csv('Data_NER.csv', index=False)

In [9]:
# Rename column
df = df.rename(columns={'entities': 'subtitle_entities'})

In [19]:
i = 0
df['keywords_entities'] = df['Keywords'].apply(lambda x: ner_lambda(x))

Exception: 'NoneType' object is not iterable
Caused by:  
Exception: 'NoneType' object is not iterable
Caused by:  


In [21]:
df.to_csv('Data_NER.csv', index=False)

In [22]:
df

,Unnamed: 0,Link,Headline,Category,Subtitle,Keywords,subtitle_entities,keywords_entities
0,0,https://www.dagbladet.no/nyheter/kaotisk-nytta...,kaotisk nyttarsnatt ute av kontroll,nyheter,Det har vært en hektisk nyttårsnatt for politi...,nyheter,{},{}
1,1,https://www.dagbladet.no/tema/fra-singel-til-d...,fra singel til damemagnet,tema,Ekspertenes beste råd til hvordan du blir attr...,"kjæreste,dating,singel",{},{}
2,2,https://www.dagbladet.no/tema/strom-nytt-fra-j...,strom nytt fra januar,tema,"Nye boligregler, bedre jobbpensjon og dyrere f...","metadz,økonomi,pensjon,skatt",{},{}
3,3,https://www.dagbladet.no/tema/anders-kvitt-mag...,anders kvitt magefettet i superfart 1,tema,"- Hvis folk bare visste hvor enkelt det er, ha...","magefett,vektnedgang,kosthold,styrketrening,",{'Anders Muren': 'PER'},{}
4,4,https://www.dagbladet.no/nyheter/taus-om-overg...,taus om overgrepsdom,nyheter,Trump ønsket «henne alt godt» i fjor sommer. N...,"donald trump,ghislaine maxwell,utenriks,nyhete...","{'Trump': 'PER', 'Ghislaine Maxwell': 'PER', '...","{'donald trump': 'PER', 'usa': 'LOC', 'jeffrey..."
...,...,...,...,...,...,...,...,...
17417,17417,https://www.dagbladet.no/sport/hjerteskjaerend...,hjerteskjaerende melding,sport,Spania og Tyskland spilte 1-1. Det betyr at sp...,"qatar-vm,sport","{'Spania': 'ORG', 'Tyskland': 'ORG', 'Qatar - ...",{}
17418,17418,https://www.dagbladet.no/nyheter/venter-stromh...,venter stromhopp i vinter,nyheter,"Lavere temperaturer og lite vind i Europa, kan...","strømpriser,været,nyheter,innenriks",{'Europa': 'LOC'},{}
17419,17419,https://www.dagbladet.no/kjendis/betatt-av-hol...,betatt av hollywood stjerne,kjendis,Ifølge ei ny bok var kong Charles litt forelsk...,"forelsket,kjendis,kong charles,barbra streisand",{'Charles': 'PER'},{}
17420,17420,https://www.dagbladet.no/tema/fra-sexlei-til-v...,fra sexlei til vanvittig tent,tema,Kvinnen ville ikke ha sex lenger. Så gjorde ma...,"xavierx,kåt,sex,",{},{}
